In [1]:
%pip install agentpy GPy GPyOpt emukit ase scikit-optimize

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Model Design
import agentpy as ap 
import pandas as pd
from boids_model import BoidsModel

import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy as cp
import random
from sklearn.model_selection import train_test_split

import ase.io
from ase.io import read, write
from ase import Atoms, Atom
from ase import units
from ase.build import molecule
from ase.md.langevin import Langevin
from ase.io.trajectory import Trajectory

import GPy
import GPyOpt
import emukit
from emukit.core import ContinuousParameter, ParameterSpace, DiscreteParameter
from emukit.core.initial_designs import RandomDesign
# from emukit.core.initial_designs.latin_design import LatinDesign
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.loop import UserFunctionResult
from emukit.core.loop import OuterLoop

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C
from skopt.space import Real
from sklearn.metrics import mean_squared_error

In [3]:
results = ap.DataDict.load(exp_name='BoidsModel', exp_id=3)

Loading from directory ap_output/BoidsModel_3/
Loading parameters_sample.csv - Successful
Loading parameters_constants.json - Successful
Loading reporters.csv - Successful
Loading info.json - Successful
Loading variables_BoidsModel.csv - Successful
Loading parameters_log.json - Successful


In [18]:
X =  results.reporters[['cohesion_strength', 'seperation_strength', 'alignment_strength', 'border_strength']]
X = X.values.tolist()

indices = list(range(len(X)))
random.shuffle(indices)

# Split indices for training and validation
split_index = int(len(indices) * 0.8)  # 80-20 split, for example
train_indices = indices[:split_index]
validate_indices = indices[split_index:]
X_train, X_val = np.array([X[i] for i in train_indices]), np.array([X[i] for i in validate_indices])
X_all = np.array(X)

In [19]:
from sklearn.preprocessing import StandardScaler

# Initialize scalers for X and Y
scaler_X = StandardScaler()

# Fit and transform the training data
X_train = scaler_X.fit_transform(X_train)

# Transform the validation and test data
X_val = scaler_X.transform(X_val)

In [78]:
num_parameters = X_all.shape[1]

# Maximum and minimum values
max_values = X_all.max(axis=0)  # Max value for each parameter across all configurations
min_values = X_all.min(axis=0)  # Min value for each parameter across all configurations

parameters_list = []
for i in range(num_parameters):
    parameter_name = f"param_{i}"
    parameters_list.append(ContinuousParameter(parameter_name, 0, 1))

# Create continuours parameter space
continuous_parameter_space = ParameterSpace(parameters_list)

In [79]:
# Parameter definitions
parameters2D = {
    'size': 50,
    'seed': 123,
    'steps': 200,
    'ndim': 2,
    'population': 200,
    'inner_radius': 5,
    'outer_radius': 12,
    'border_distance': 15,
    'cohesion_strength': 0.005,
    'seperation_strength': 0.1,
    'alignment_strength': 0.3,
    'border_strength': 0.5
}

In [89]:
target = float(input("Enter target flock_density value: "))

In [90]:
def f(knobs_arr):
    mse_arr = []
    for knobs in knobs_arr:
        parameters_multi = dict(parameters2D)
        parameters_multi.update({
            'cohesion_strength': knobs[0],
            'seperation_strength': knobs[1],
            'alignment_strength': knobs[2],
            'border_strength': knobs[3]
        })
        sim = BoidsModel(parameters_multi)
        outputs = sim.run()
        result = outputs.reporters[['flock_density']].values.tolist()
        mse_arr.append(mean_squared_error([target], result))

    return np.array(mse_arr).reshape(-1, 1)

In [91]:
design = RandomDesign(continuous_parameter_space) # Collect random points
num_data_points = 10
X = design.get_samples(num_data_points)
Y = f(X)

Completed: 200 steps
Run time: 0:00:07.015997
Simulation finished
Completed: 200 steps
Run time: 0:00:10.567027
Simulation finished
Completed: 200 steps
Run time: 0:00:09.330644
Simulation finished
Completed: 200 steps
Run time: 0:00:05.457261
Simulation finished
Completed: 200 steps
Run time: 0:00:09.579813
Simulation finished
Completed: 200 steps
Run time: 0:00:10.915970
Simulation finished
Completed: 200 steps
Run time: 0:00:06.564475
Simulation finished
Completed: 200 steps
Run time: 0:00:12.872683
Simulation finished
Completed: 200 steps
Run time: 0:00:06.109036
Simulation finished
Completed: 200 steps
Run time: 0:00:05.672393
Simulation finished


In [92]:
model_gpy = GPRegression(X,Y) # Train and wrap the model in Emukit
model_emukit = GPyModelWrapper(model_gpy)
expected_improvement = ExpectedImprovement(model = model_emukit)

bayesopt_loop = BayesianOptimizationLoop(model = model_emukit,
                                         space = continuous_parameter_space,
                                         acquisition = expected_improvement,
                                         batch_size = 1)

In [93]:
max_iterations = 100
bayesopt_loop.run_loop(f, max_iterations)

Completed: 200 steps
Run time: 0:00:05.829015
Simulation finished
Completed: 200 steps
Run time: 0:00:05.694443
Simulation finished
Completed: 200 steps
Run time: 0:00:05.699392
Simulation finished
Completed: 200 steps
Run time: 0:00:05.908922
Simulation finished
Completed: 200 steps
Run time: 0:00:05.793792
Simulation finished
Completed: 200 steps
Run time: 0:00:05.615482
Simulation finished
Completed: 200 steps
Run time: 0:00:05.661175
Simulation finished
Completed: 200 steps
Run time: 0:00:05.706674
Simulation finished
Completed: 200 steps
Run time: 0:00:06.132004
Simulation finished
Completed: 200 steps
Run time: 0:00:05.687440
Simulation finished
Completed: 200 steps
Run time: 0:00:05.682340
Simulation finished
Completed: 200 steps
Run time: 0:00:05.764438
Simulation finished
Completed: 200 steps
Run time: 0:00:08.514309
Simulation finished
Completed: 200 steps
Run time: 0:00:05.771698
Simulation finished
Completed: 200 steps
Run time: 0:00:06.581022
Simulation finished
Completed:

In [94]:
results = bayesopt_loop.get_results()

In [95]:
print(results.minimum_value)

0.00022678286904710378


In [96]:
print(results.minimum_location)

[0.0077725  0.93259337 0.17260873 0.09502598]


In [97]:
best_location = results.minimum_location
parameters_multi = dict(parameters2D)
parameters_multi.update({
    'cohesion_strength': best_location[0],
    'seperation_strength': best_location[1],
    'alignment_strength': best_location[2],
    'border_strength': best_location[3]
})
sim = BoidsModel(parameters_multi)
outputs = sim.run()
result = outputs.reporters[['flock_density']]
print(result)

Completed: 200 steps
Run time: 0:00:05.793701
Simulation finished
   flock_density
0       0.075059
